## import dependencies

In [1]:
from transformers import pipeline, set_seed

import matplotlib.pyplot as plt
from datasets import load_dataset
import pandas as pd
from datasets import load_dataset, load_metric, concatenate_datasets, DatasetDict

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

import nltk
from nltk.tokenize import sent_tokenize

from tqdm import tqdm
import torch

nltk.download("punkt")

/home/p/miniconda3/envs/nlp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to /home/p/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DEVICE
PRETRAINED_MODEL_PATH = "google/pegasus-cnn_dailymail"
TOKENIZER = AutoTokenizer.from_pretrained(PRETRAINED_MODEL_PATH)
PRETRAINED_MODEL = AutoModelForSeq2SeqLM.from_pretrained(PRETRAINED_MODEL_PATH).to(DEVICE)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
samsum_data = load_dataset("samsum")
legal_summarization_data = load_dataset("lighteval/legal_summarization","BillSum")
samsum_data, legal_summarization_data

(DatasetDict({
     train: Dataset({
         features: ['id', 'dialogue', 'summary'],
         num_rows: 14732
     })
     test: Dataset({
         features: ['id', 'dialogue', 'summary'],
         num_rows: 819
     })
     validation: Dataset({
         features: ['id', 'dialogue', 'summary'],
         num_rows: 818
     })
 }),
 DatasetDict({
     train: Dataset({
         features: ['article', 'summary'],
         num_rows: 18949
     })
     test: Dataset({
         features: ['article', 'summary'],
         num_rows: 3269
     })
 }))

In [4]:
sam_split_len = [len(samsum_data[split])for split in samsum_data]
legal_split_len = [len(legal_summarization_data[split])for split in legal_summarization_data]
samsum_data = samsum_data.remove_columns('id').rename_column("dialogue","article")

In [5]:
def merge_datasets(ds):
  merged_dataset = DatasetDict()
  merged_dataset['train'] = concatenate_datasets([dataset['train'] for dataset in ds],split="train")
  merged_dataset['test'] = concatenate_datasets([dataset['test'] for dataset in ds],split="test")
  merged_dataset['validation'] = concatenate_datasets([dataset['validation'] for dataset in ds if len(dataset)==3])
  return merged_dataset
dataset = merge_datasets([legal_summarization_data,samsum_data])

In [34]:
article = dataset['test'][250]['article']
article = article[:128]
len(article.split(" ")), article

(22,
 "SECTION 1. SHORT TITLE. This Act may be cited as the ``Federal Energy Price Protection Act''. SEC. 2. PROTECTION OF CONSUMERS AG")

In [24]:
len(article.split(" "))

1291

In [30]:
pipe = pipeline('summarization', model=PRETRAINED_MODEL,tokenizer=TOKENIZER,)

In [35]:
pipe_out = pipe(article)
pipe_out

Your max_length is set to 128, but your input_length is only 33. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=16)


[{'summary_text': "This Act may be cited as the Federal Energy Price Protection Act''<n>This Act may be cited as the Federal Energy Price Protection Act''"}]

In [36]:
rouge_metric = load_metric('rouge')
score = calcu

/tmp/ipykernel_18460/1465679614.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge_metric = load_metric('rouge')
/home/p/miniconda3/envs/nlp/lib/python3.10/site-packages/datasets/load.py:759: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [38]:
TOKENIZER(article,)

{'input_ids': [67794, 3898, 62268, 54375, 107, 182, 2046, 218, 129, 8919, 130, 109, 110, 105, 30983, 3013, 3544, 4872, 2046, 131, 131, 107, 11257, 107, 280, 107, 90154, 2793, 12852, 67550, 11862, 11357, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}